Corpus (be.txt) taken from CC100 dataset: https://metatext.io/datasets/cc100-belarusian

In [16]:
import re
import gensim
from gensim.parsing.preprocessing import strip_punctuation, strip_short, strip_numeric, strip_multiple_whitespaces, remove_stopwords
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

In [17]:
import logging
logging.basicConfig(level=logging.INFO, force = True)
logger = logging.getLogger()
logger.info("Logging initialized")

INFO:root:Logging initialized


In [68]:
import xml.dom.minidom
from itertools import islice

def calculate_mapping_from_forms_to_base(filepath):
    xml_doc = xml.dom.minidom.parse(filepath)
    words = xml_doc.getElementsByTagName('Variant')
    result = {}
    collision_count = 0
    collisions = set()
    for word in words:
        base = word.getAttribute('lemma').replace("+", "")
        forms_objs = word.getElementsByTagName('Form')
        local_map = {}
        citation_count = max([forms_obj.getAttribute('slouniki').count(',') for forms_obj in forms_objs]) + 1
        for forms_obj in forms_objs:
            if (len(forms_obj.childNodes) == 0):
                logger.info(f"Empty for {base}")
            else:
                form = forms_obj.childNodes[0].data.replace("+", "")
                #if citation_count > 3:
                local_map[form] = (base, citation_count)
        for k, v in local_map.items():
            if k in result:
                if result[k][1] == v[1] and result[k][0] != v[0]:
                    collision_count += 1
                    collisions.add(v[0])
                    collisions.add(result[k][0])
                elif result[k][1] < v[1]:
                    result[k] = v
            else:
                result[k] = v
        #result.update(local_map)
    logger.info(f"Collisions (forms leading to different base word, and having same amount of citation): {collision_count}")
    logger.info(f"Examples of collisions: {list(islice(collisions, 5))}")
    for k in result:
        result[k] = result[k][0]
    return result

In [69]:
n1 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/N1.xml')
n2 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/N2.xml')
n3 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/N3.xml')
adj1 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/A1.xml')
adj2 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/A2.xml')

WORD_MAP = {}
WORD_MAP.update(n1)
WORD_MAP.update(n2)
WORD_MAP.update(n3)
WORD_MAP.update(adj1)
WORD_MAP.update(adj2)

INFO:root:Empty for бібікі
INFO:root:Empty for бібікі
INFO:root:Empty for бібікі
INFO:root:Empty for бібікі
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 1345
INFO:root:Examples of collisions: ['імерэцінка', 'жаўталіст', 'гераін', 'века', 'абісінец']
INFO:root:Empty for лапта
INFO:root:Empty for мальба
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 1155
INFO:root:Examples of collisions: ['мальдыўка', 'неспадзеў', 'корм', 'конь', 'падыспытная']
INFO:root:Empty for тамада
INFO:root:Empty for тамада
INFO:root:Empty for фата
INFO:root:Empty for чака
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 954
INFO:root:Examples of collisions: ['тальк', 'цэфалаподы', 'развалы', 'сума', 'страйкам']
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 142
INFO:root:Examples of collisions: ['вітавы', 'крэчатавы'

In [70]:
print(len(WORD_MAP))

1567761


In [71]:
WORD_MAP['рухам']

'рух'

In [72]:
def strip_trailing_newline(iterable):
    for i in iterable:
        yield i.rstrip()

CHARACTERS_MAP = {'ý': 'ў', 'i': 'і', 'ньн': 'нн', 'цьц': 'цц', 'сьц': 'сц', 'сьл':'сл'}
def replace_characters(iterable):
    for s in iterable:
        for k, v in CHARACTERS_MAP.items():
            s = s.replace(k, v)
        yield s

def split_sentences(iterable):
    for i in iterable:
        merged_dots = re.sub("[\.]+", ".", i)
        sentences = merged_dots.split('.')
        for s in sentences:
            yield s

STOPWORDS = ['на', 'не', 'што', 'да', 'па', 'як', 'за', 'для', 'гэта', 'ад', 'але', 'калі', 'пра', 'у', 'яго', 'якія', 'ён', 'іх', 'мы', 'каб', 'толькі', 'аб', 'таксама', 'які', 'ці', 'быў', 'было', 'яны', 'так', 'вы', 'яе', 'будзе', 'пры', 'яшчэ', 'тым', 'таму', 'вельмі', 'былі', 'можна', 'яна', 'пасля', 'пад', 'можа', 'дзе', 'якая', 'тут', 'была', 'трэба', 'тое', 'таго', 'або', 'гэтым', 'бо', 'ўсё', 'хто', 'ня', 'нас', 'гэтага', 'быць', 'гэты', 'ёсць', 'праз', 'ўжо', 'са', 'нават', 'то', 'мяне', 'ім','ва', 'той', 'усе', 'без', 'чым', 'мне', 'мае', 'сябе', 'гэтай', 'там', 'усё', 'вось', 'ды', 'каля', 'якіх', 'ты', 'якой', 'ўсе', 'жа', 'ужо', 'паводле', 'будуць', 'аднак', 'могуць', 'сваю', 'ні', 'сваёй', 'яму', 'свае', 'гэтыя', 'проста', 'ўсіх', 'якім', 'падчас', 'тады', 'свой', 'вас', 'паміж', 'нам', 'раз', 'сваіх', 'нашай', 'менавіта', 'перад', 'вам', 'тых','зь','такім', 'свайго', 'над', 'зараз', 'амаль', 'чаму', 'ёй', 'чынам', 'напрыклад', 'якога', 'якое', 'сваім', 'можаце', 'га', 'хоць', 'бы', 'тыя', 'такія', 'потым', 'адным', 'такі', 'якую', 'сабе','сам','гэтых','мая','наш','зусім','чаго','наша','зноў','дык','такіх','нашага','адразу','каго','самі','ст','ну','нашы','нашым','самы','яно','гэтае','дзеля','куды','by','гг']
def preprocess_sentences(iterable):
    for i in iterable:
        s = strip_multiple_whitespaces(strip_numeric(strip_short(strip_punctuation(i))))
        s = s.lower()
        s = re.sub("[«»“”„…—°′²]", "", s)
        s = remove_stopwords(s, stopwords=STOPWORDS)
        s = ' '.join([WORD_MAP.get(w, w) for w in s.split()])
        yield s

def remove_short_lines(iterable):
    for i in iterable:
        if not i.isspace() and len(i) >= 20:
            yield i


In [73]:
with open('be.txt', 'r') as original_file:
    with open('sentences.txt', 'w') as sentences_file:
        for s in remove_short_lines(preprocess_sentences(split_sentences(replace_characters(strip_trailing_newline(original_file))))):
            sentences_file.write(s + "\n")

In [74]:
loss_list = []
class Callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_list.append(loss)
        print('Loss after epoch {}:{}'.format(self.epoch, loss))
        model.running_training_loss = 0.0
        self.epoch = self.epoch + 1

In [75]:
model = Word2Vec(vector_size=100, window=3, min_count=10, workers=5)

INFO:gensim.utils:Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-04-22T18:18:42.284078', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'created'}


In [76]:
sentences = LineSentence('sentences.txt')

In [77]:
model.build_vocab(sentences, progress_per=5000000)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #5000000, processed 44754492 words, keeping 895623 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000000, processed 89584467 words, keeping 1350126 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #15000000, processed 134296356 words, keeping 1699597 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000000, processed 179110867 words, keeping 1997894 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #25000000, processed 223797301 words, keeping 2271233 word types
INFO:gensim.models.word2vec:collected 2286211 word types from a corpus of 226493880 raw words and 25306752 sentences
INFO:gensim.models.word2vec:Creating a fresh vocabulary
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 328798 unique

In [78]:
model.train(sentences, epochs=50, total_examples=model.corpus_count, total_words=model.corpus_total_words, compute_loss=True, report_delay=120, callbacks=[Callback()])


INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training model with 5 workers on 328798 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=3 shrink_windows=True', 'datetime': '2023-04-22T18:20:09.631800', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'train'}
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 0.58% examples, 1265770 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 47.87% examples, 871369 words/s, in_qsize 10, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 94.48% examples, 861497 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 0: training on 226493880 raw words (220063745 effective words) took 250.9s, 877088 effective words/s


Loss after epoch 0:45268440.0


INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 0.57% examples, 1243133 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 52.18% examples, 949579 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1: training on 226493880 raw words (220060555 effective words) took 230.1s, 956426 effective words/s


Loss after epoch 1:43138140.0


INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 0.50% examples, 89227 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 48.43% examples, 806345 words/s, in_qsize 8, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 98.56% examples, 859817 words/s, in_qsize 9, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 2: training on 226493880 raw words (220059964 effective words) took 254.7s, 864026 effective words/s


...

INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 0.60% examples, 1295986 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 51.80% examples, 942646 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 48: training on 226493880 raw words (220061289 effective words) took 229.5s, 959054 effective words/s


Loss after epoch 48:37483072.0


INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 0.55% examples, 116964 words/s, in_qsize 8, out_qsize 2
INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 53.21% examples, 899569 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 49: training on 226493880 raw words (220060630 effective words) took 242.2s, 908436 effective words/s
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training on 11324694000 raw words (11003047035 effective words) took 11771.5s, 934723 effective words/s', 'datetime': '2023-04-22T23:13:31.550670', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'train'}


Loss after epoch 49:37343080.0


(11003047035, 11324694000)

In [87]:
model.save("word2vec-100-bel-cc100.model")

INFO:gensim.utils:Word2Vec lifecycle event {'fname_or_handle': 'word2vec-100-bel-cc100.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-04-23T01:10:15.447782', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'saving'}
INFO:gensim.utils:storing np array 'vectors' to word2vec-100-bel-cc100.model.wv.vectors.npy
INFO:gensim.utils:storing np array 'syn1neg' to word2vec-100-bel-cc100.model.syn1neg.npy
INFO:gensim.utils:not storing attribute cum_table
INFO:gensim.utils:saved word2vec-100-bel-cc100.model


In [81]:
model.wv.most_similar('воўк', topn=20)

[('мядзведзь', 0.7393637895584106),
 ('звер', 0.7242729067802429),
 ('заяц', 0.7042849063873291),
 ('бык', 0.6875906586647034),
 ('муха', 0.6802892684936523),
 ('драпежнік', 0.6730962991714478),
 ('кабан', 0.6689190864562988),
 ('алень', 0.6683720350265503),
 ('выгаладаўся', 0.656315803527832),
 ('варона', 0.6556535363197327),
 ('змей', 0.6529685854911804),
 ('сабака', 0.6515656113624573),
 ('табун', 0.6455168724060059),
 ('драпежны', 0.6449723243713379),
 ('мураш', 0.6393277049064636),
 ('лісіца', 0.633025586605072),
 ('жаба', 0.6329237818717957),
 ('курапатка', 0.6244716644287109),
 ('вуж', 0.6240508556365967),
 ('конь', 0.6231964230537415)]

In [82]:
model.wv.index_to_key

['год',
 'беларускі',
 'чалавек',
 'беларусі',
 'час',
 'дзень',
 'больш',
 'мова',
 'новы',
 'беларусь',
 'вялікі',
 'краіна',
 'жыццё',
 'праца',
 'горад',
 'месца',
 'справа',
 'гісторыя',
 'рэспубліка',
 'слова',
 'кожны',
 'гульня',
 'дзіцё',
 'дзяржаўны',
 'сайт',
 'нацыянальны',
 'раён',
 'пытанне',
 'імя',
 'галоўны',
 'свет',
 'культура',
 'кніга',
 'старонка',
 'раз',
 'другі',
 'розны',
 'адзін',
 'арганізацыя',
 'школа',
 'апошні',
 'дом',
 'цэнтр',
 'матэрыял',
 'вынік',
 'правіць',
 'праект',
 'цяпер',
 'беларус',
 'права',
 'адукацыя',
 'вайна',
 'частка',
 'праграма',
 'дзяржава',
 'зямля',
 'шмат',
 'міжнародны',
 'сёння',
 'гад',
 'работа',
 'інфармацыя',
 'развіццё',
 'выбар',
 'няма',
 'лепшы',
 'народны',
 'наступны',
 'палітычны',
 'іншых',
 'праблема',
 'пачатак',
 'вёска',
 'народ',
 'кіраўнік',
 'мяжа',
 'стол',
 'літаратура',
 'артыкул',
 'бацька',
 'дзейнасць',
 'магчымасць',
 'выпадак',
 'дапамога',
 'бок',
 'вобласць',
 'асноўны',
 'выкарыстанне',
 'сябар',

In [37]:
model.wv.get_vecattr("чарадзей", "count")

1379